# Customer Segmentation and Recommendation System Project

## Introduction

In this project, we aim to enhance marketing strategies and increase sales for an online retail business by analyzing transactional data from a UK-based retailer. The dataset, which spans from 2010 to 2011, contains detailed information about customer purchases, including transaction dates, product descriptions, quantities, and prices.

### Project Objectives

1. **Customer Segmentation**: To group customers into distinct segments based on their purchasing behavior using the K-means clustering algorithm. This segmentation will allow the business to tailor marketing strategies to different customer groups effectively.

2. **Recommendation System**: To develop a recommendation system that suggests top-selling products to customers within each segment who have not yet purchased those items. This will help boost sales and enhance customer satisfaction by providing personalized product recommendations.

### Step 1: Data Understanding and Exploration

In the first step of this project, we will explore the dataset to understand its structure, identify any data quality issues, and gain insights into the key features that will be used for customer segmentation and recommendation. The main tasks in this step include:

- Loading the dataset into a pandas DataFrame.
- Performing an initial inspection to understand the structure and content of the data.
- Checking for missing values and data types to ensure data quality.
- Generating basic statistics to understand the distribution and characteristics of numerical and categorical features.

By thoroughly understanding the dataset, we will lay the foundation for effective data preprocessing, feature engineering, and model development in the subsequent steps of the project.

In [5]:
# Import necessary libraries
import pandas as pd

In [6]:
# Load the dataset
file_path = 'data/retail-data.csv'
df = pd.read_csv(file_path)

In [7]:
# Display the first few rows of the dataset to understand its structure
print("First 5 rows of the dataset:")
display(df.head())

First 5 rows of the dataset:


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
